## 과제 1
ReLu activation function과 derivative function을 구현해보세요
- Hint : np.maximum 함수 사용하면 편리합니다
- 다른 방법 사용하셔도 무방합니다


In [1]:
import numpy as np

In [2]:
def relu(x):
  return(np.maximum(0,x))

In [3]:
def d_relu(x):
  x = np.maximum(0,x)
  
  if x > 0 :
    return 1
  else:
    return 0

## 과제 2
Deep Learning Basic 코드 파일의 MLP implementation with Numpy library using MNIST dataset 코드 참고해서
Three layer MLP 일 때의 backward_pass 함수를 완성해주세요.   
- Hint : 코드 파일의 예시는 Two layer MLP


In [ ]:
def foward_pass(x, params):
  
  params["S1"] = np.dot(params["W1"], x) + params["b1"]
  params["A1"] = sigmoid(params["S1"])
  params["S2"] = np.dot(params["W2"], params["A1"]) + params["b2"]
  params["A2"] = sigmoid(params["S2"])
  params["S3"] = np.dot(params["W3"], params["A2"]) + params["b3"]
  params["A3"] = softmax(params["S3"])

  return params

In [ ]:
def backward_pass(x, y_true, params):

  dS3 = params["A3"] - y_true

  grads = {}

  grads["dW3"] =  np.dot(dS3, params["A2"].T)/x.shape[1]
  grads["db3"] =  (1/x.shape[1])*np.sum(dS3, axis=1, keepdims=True)/x.shape[1]

  dA2 = np.dot(params["W3"].T, dS3)
  dS2 = dA2 * d_sigmoid(params["S2"])

  grads["dW2"] = np.dot(dS2, x.T)/x.shape[1]
  grads["db2"] = np.sum(dS2, axis=1, keepdims=True)/x.shape[1]

  dA1 = np.dot(params["W2"].T, dS2)
  dS1 = dA1 * d_sigmoid(params["S1"])

  grads["dW1"] = np.dot(dS1, x.T)/x.shape[1]
  grads["db1"] = np.sum(dS1, axis=1, keepdims=True)/x.shape[1]

  return grads

## 과제 3
Deep Learning Basic 코드 파일의 MLP implementation with Pytorch library using MNIST dataset 코드 참고해서
Three layer MLP를 구한후, 학습을 돌려 보세요

hyperparameter는 다음과 같이 설정

- epochs : 100
- hiddensize : 128, 64 (two layer)
- learning_rate : 0.5

In [ ]:
# Assignment 3 구현은 여기서 ()

In [18]:
from torchvision import transforms, datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [19]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [20]:
trainset = datasets.MNIST(
    root      = './.data/', 
    train     = True,
    download  = True,
    transform = transform
)
testset = datasets.MNIST(
    root      = './.data/', 
    train     = False,
    download  = True,
    transform = transform
)

In [21]:
BATCH_SIZE = 512
# train set과 test set 각각에 대하여 DataLoader를 생성합니다.
# shuffle=True 매개변수를 넣어 데이터를 섞어주세요.
train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
test_loader =  DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)

In [22]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(784,128)
        self.layer2 = nn.Linear(128,64)
        self.layer3 = nn.Linear(64,10)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(-1, 784)
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        out = self.relu(out)
        out = self.layer3(out)

        return out

In [23]:
model = Net()
model

Net(
  (layer1): Linear(in_features=784, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)

In [24]:
list(model.parameters()) # 행렬들을 직접 살펴볼 수 있음
                         # require_true 얘는 학습되는 애구나 알 수 있음

[Parameter containing:
 tensor([[ 0.0215, -0.0170, -0.0219,  ..., -0.0127, -0.0134, -0.0280],
         [-0.0331, -0.0222,  0.0274,  ..., -0.0044,  0.0231,  0.0206],
         [-0.0243,  0.0201, -0.0031,  ...,  0.0257, -0.0089, -0.0073],
         ...,
         [-0.0304,  0.0011,  0.0344,  ..., -0.0203,  0.0129, -0.0159],
         [ 0.0197,  0.0098,  0.0253,  ...,  0.0134, -0.0252,  0.0231],
         [-0.0239, -0.0064,  0.0247,  ..., -0.0032, -0.0143,  0.0107]],
        requires_grad=True), Parameter containing:
 tensor([-0.0024, -0.0061, -0.0159, -0.0241, -0.0247, -0.0356, -0.0003,  0.0096,
          0.0102, -0.0121, -0.0142, -0.0266, -0.0295,  0.0263, -0.0253, -0.0082,
          0.0179,  0.0345, -0.0144,  0.0356,  0.0055, -0.0272, -0.0312, -0.0056,
         -0.0352, -0.0043, -0.0214,  0.0311, -0.0288, -0.0297, -0.0075,  0.0323,
          0.0074,  0.0138, -0.0167,  0.0166,  0.0193, -0.0270, -0.0285, -0.0141,
          0.0032, -0.0115, -0.0064,  0.0252, -0.0177, -0.0025,  0.0096, -0.0190,

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.5)

In [26]:
def train(model, train_loader, optimizer):
    model.train()
    # 배치 당 loss 값을 담을 리스트 생성
    batch_losses = []

    for data, target in train_loader:
        # 옵티마이저의 기울기 초기화
        optimizer.zero_grad()

        # y pred 값 산출
        output = model(data)
        # loss 계산
        # 정답 데이터와의 cross entropy loss 계산
        # 이 loss를 배치 당 loss로 보관
        loss = criterion(output, target)
        batch_losses.append(loss)

        # 기울기 계산
        loss.backward()

        # 가중치 업데이트!
        optimizer.step()
        
    # 배치당 평균 loss 계산
    avg_loss = sum(batch_losses) / len(batch_losses)
    
    return avg_loss

In [27]:
def evaluate(model, test_loader):
    # 모델을 평가 모드로 전환
    model.eval()

    batch_losses = []
    correct = 0 

    with torch.no_grad(): 
        for data, target in test_loader:
            # 예측값 생성
            output = model(data)

            # loss 계산 (이전과 동일)
            loss = criterion(output, target)
            batch_losses.append(loss)

           # Accuracy 계산
           # y pred와 y가 일치하면 correct에 1을 더해주기
            pred = output.max(1, keepdim=True)[1]

            # eq() 함수는 값이 일치하면 1을, 아니면 0을 출력.
            correct += pred.eq(target.view_as(pred)).sum().item()

    # 배치 당 평균 loss 계산 
    avg_loss =  sum(batch_losses) / len(batch_losses)

    #정확도 계산
    accuracy = 100. * correct / len(test_loader.dataset)

    return avg_loss, accuracy

In [28]:
EPOCHS = 100

for epoch in range(1, EPOCHS + 1):
    train_loss = train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Train Loss: {:.4f}\tTest Loss: {:.4f}\tAccuracy: {:.2f}%'.format(
          epoch, train_loss, test_loss, test_accuracy))

[1] Train Loss: 0.7955	Test Loss: 0.2814	Accuracy: 91.49%
[2] Train Loss: 0.2360	Test Loss: 0.1890	Accuracy: 94.18%
[3] Train Loss: 0.1646	Test Loss: 0.1554	Accuracy: 95.03%
[4] Train Loss: 0.1310	Test Loss: 0.1586	Accuracy: 95.06%
[5] Train Loss: 0.1080	Test Loss: 0.2659	Accuracy: 91.72%
[6] Train Loss: 0.2214	Test Loss: 0.1540	Accuracy: 95.11%
[7] Train Loss: 0.0964	Test Loss: 0.1077	Accuracy: 96.69%
[8] Train Loss: 0.0806	Test Loss: 0.1049	Accuracy: 96.65%
[9] Train Loss: 0.0695	Test Loss: 0.1000	Accuracy: 96.91%
[10] Train Loss: 0.0611	Test Loss: 0.0932	Accuracy: 96.91%
[11] Train Loss: 0.0544	Test Loss: 0.1058	Accuracy: 96.79%
[12] Train Loss: 0.0491	Test Loss: 0.2320	Accuracy: 93.76%
[13] Train Loss: 0.0467	Test Loss: 0.0863	Accuracy: 97.29%
[14] Train Loss: 0.0383	Test Loss: 0.0746	Accuracy: 97.60%
[15] Train Loss: 0.0360	Test Loss: 0.0795	Accuracy: 97.58%
[16] Train Loss: 0.0319	Test Loss: 0.0743	Accuracy: 97.72%
[17] Train Loss: 0.0288	Test Loss: 0.0942	Accuracy: 97.39%
[18] T

## 과제 4
과제 3 부분의 성능을 지금까지 배운 지식을 바탕으로 향상시켜보세요

- Hint : Activation function, hyperparameter setting

In [ ]:
# Assignment 4 구현은 여기서 ()

In [43]:
BATCH_SIZE = 512
# train set과 test set 각각에 대하여 DataLoader를 생성합니다.
# shuffle=True 매개변수를 넣어 데이터를 섞어주세요.
train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
test_loader =  DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)

In [51]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(784,128)
        self.layer2 = nn.Linear(128,64)
        self.layer3 = nn.Linear(64,10)
        self.relu = nn.LeakyReLU(0.1)
        
    def forward(self, x):
        x = x.view(-1, 784)
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        out = self.relu(out)
        out = self.layer3(out)

        return out

In [52]:
model2 = Net()
model2

Net(
  (layer1): Linear(in_features=784, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=10, bias=True)
  (relu): LeakyReLU(negative_slope=0.1)
)

In [56]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model2.parameters(), lr=0.05)

In [58]:
EPOCHS = 100

for epoch in range(1, EPOCHS + 1):
    train_loss = train(model2, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model2, test_loader)
    
    print('[{}] Train Loss: {:.4f}\tTest Loss: {:.4f}\tAccuracy: {:.2f}%'.format(
          epoch, train_loss, test_loss, test_accuracy))

[1] Train Loss: 0.9551	Test Loss: 0.6251	Accuracy: 83.67%
[2] Train Loss: 0.5370	Test Loss: 0.4497	Accuracy: 87.25%
[3] Train Loss: 0.4263	Test Loss: 0.3835	Accuracy: 89.16%
[4] Train Loss: 0.3775	Test Loss: 0.3591	Accuracy: 89.80%
[5] Train Loss: 0.3488	Test Loss: 0.3246	Accuracy: 90.58%
[6] Train Loss: 0.3280	Test Loss: 0.3088	Accuracy: 91.09%
[7] Train Loss: 0.3119	Test Loss: 0.2983	Accuracy: 91.23%
[8] Train Loss: 0.2993	Test Loss: 0.2832	Accuracy: 91.72%
[9] Train Loss: 0.2860	Test Loss: 0.2719	Accuracy: 92.15%
[10] Train Loss: 0.2770	Test Loss: 0.2656	Accuracy: 92.26%
[11] Train Loss: 0.2668	Test Loss: 0.2730	Accuracy: 92.22%
[12] Train Loss: 0.2576	Test Loss: 0.2484	Accuracy: 93.07%
[13] Train Loss: 0.2501	Test Loss: 0.2454	Accuracy: 92.81%
[14] Train Loss: 0.2411	Test Loss: 0.2390	Accuracy: 92.82%
[15] Train Loss: 0.2332	Test Loss: 0.2285	Accuracy: 93.43%
[16] Train Loss: 0.2255	Test Loss: 0.2197	Accuracy: 93.46%
[17] Train Loss: 0.2186	Test Loss: 0.2120	Accuracy: 93.75%
[18] T

**무엇을 보완하였고, 왜 보완되었는지에 대한 자유 서술 (아래에)**

1. 활성화함수의 경우, ReLU를 보완한 LeakyReLU를 사용하였다.
2. 학습률을 0.5에서 0.05로 바꾸었다.
3. Net 모델에서는 굳이 layer를 하나 더 쌓을 필요 없다고 생각하였다. 왜냐하면, 이미 과제3에서 정확도가 97.97%가 나왔기 때문에 layer의 보완보다는 hyperparameter의 보완이 유용할 수 있다고 판단하였다.
4. BUT, 정확도의 경우 약 0.5%정도 감소한 결과가 나왔다... 하지만, test loss가 작게 나와서 의미가 없지 않다고 생각한다.